In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
PI_table = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table.nodes_PI = PI_table.nodes_PI + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:4])
First_Five.index = First_Five.index + 1

In [4]:
#return the total power of node i.
def TPN(i):
    PI_ofi = Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    n1=TC_ofi /Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n2=PI_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n3=CC_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    #equation []
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    
    return TPN_i

In [5]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [6]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1])

In [7]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(Top_Five)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

   nodes_TPN     TPN_i
1         34  0.738881
2         33  1.343098
3          1  2.108931
4         32  2.959926
5          2  3.457105


In [8]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_Five = pd.DataFrame(Propose_Result_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
#convert column nodes_TPN to int
Top_Five.nodes_TPN= Top_Five.nodes_TPN.astype(int)
print(Top_Five)
#Propose_Table.to_csv('result/Propose_Table_kc_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC    TC_i  nodes_PI    PI_i  nodes_CC      CC_i  nodes_TPN     TPN_i
1         0  0.5781         2 -0.9069         0  0.155423         34  0.738881
2        33  0.5862         8 -0.8722        33  0.156419         33  1.343098
3        32  0.6000         7 -0.8591         2  0.185177          1  2.108931
4         1  0.6250        13 -0.8591        31  0.244575         32  2.959926
5         2  0.6389         3 -0.8346        32  0.249223          2  3.457105
